# Übung 8 Neuronal Network - Rainier Robles & Valentin Wolf

In [609]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [704]:
def transform_y(y,total_classes=10):
    """gets 1-D vector with the classes (all positive and consecutive)
    and converts it to an matrix of shape (classes,Ys) 
    with a 1 at where the class is and 0s everywhere else"""
    y=y.astype('int8')
    if total_classes == None:
        total_classes = np.unique(y)
        
    new_y = np.zeros((y.shape[0],total_classes))
    for i in range(y.shape[0]):
        new_y[i,y[i]] = 1
    return new_y


In [706]:
train = pd.read_table('zip.train', delim_whitespace=True,header=None)
test = pd.read_table('zip.test', delim_whitespace=True,header=None)

#split labels y from data X
y_train = transform_y(train[0].as_matrix(),total_classes=10)
y_tt = train[0].as_matrix()
X_train = train.drop(0, axis=1).as_matrix()

y_test = transform_y(test[0].as_matrix(),total_classes=10)
X_test = test.drop(0, axis=1).as_matrix()

In [725]:
class Classifier():
    def error_rate(self,truth, pred):
        """gets two vectors, returns (wrongly classified / total)"""
        return 1 - self.accuracy(truth, pred)
    
    def accuracy(self,truth,pred):
        return 1 - (np.abs((truth-pred)).sum(axis=1)/2).mean()# np.mean(truth == pred)

In [728]:
class NeuronalNetwork(Classifier):
    def __init__(self,sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.acc_rates = []
    
    def train(self, X, y, epochs=10, learning_rate=0.5, printfreq=25, X_test=None,y_test=None):
        """
        X: datapoints
        y: lables as rows in matrix with 1 when class and 0 else 
        """
        learning_rate = learning_rate/X.shape[0]
        for i in range(epochs):
            if i % printfreq == 0 or i == epochs-1:
                try:
                    print("Iteration {} | Training Accuracy: {} | Test Accuracy: {}".format(
                                i,self.predict_acc(X,y),self.predict_acc(X_test,y_test)))
                    self.acc_rates.append(self.predict_acc(X_test,y_test))
                except AttributeError: 
                    print("Iteration {} | Training Accuracy: {}".format(i,self.predict_acc(X,y)))

            for j in range(X.shape[0]):
                deltas_b, deltas_w = self.backprop(X[j][np.newaxis],y[j])
                self.weights = [w-(learning_rate*dw)
                            for w, dw in zip(self.weights, deltas_w)] 
                self.biases = [(b-(learning_rate*db) )
                            for b, db in zip(self.biases, deltas_b)]
    
    def backprop(self,x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x.T
        activations = [x.T] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta     #np.sum(delta.T,axis=0)[np.newaxis].T
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            #np.sum(delta.T,axis=0)[np.newaxis].T
            #print(nabla_b[-l].shape)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        output_activations -= y[np.newaxis].T

        return (output_activations) 
    
    def sigmoid(self,z):
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoid_deriv(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
        
    def feedforward(self,X):
        a = X.T
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a
    
    def predict(self,X):
        col_y = np.argmax(self.feedforward(X),axis=0)
        return transform_y(col_y,total_classes=10)
    
    def predict_error(self,X,y):
        return self.error_rate(self.predict(X),y)
    def predict_acc(self,X,y):
        return 1 - self.predict_error(X,y)

In [640]:
y_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [731]:
NN = NeuronalNetwork([256,30,10])
NN.train(X_train,y_train,epochs=100,learning_rate=300,X_test=X_test,y_test=y_test,printfreq=1)

Iteration 0 | Training Accuracy: 0.22767795912769162 | Test Accuracy: 0.22919780767314402
Iteration 1 | Training Accuracy: 0.6069126320120697 | Test Accuracy: 0.5635276532137519
Iteration 2 | Training Accuracy: 0.749691400356604 | Test Accuracy: 0.6970602889885401
Iteration 3 | Training Accuracy: 0.7880949115347688 | Test Accuracy: 0.7354260089686099
Iteration 4 | Training Accuracy: 0.8107255520504733 | Test Accuracy: 0.7538614848031888
Iteration 5 | Training Accuracy: 0.8221094500068578 | Test Accuracy: 0.7668161434977578
Iteration 6 | Training Accuracy: 0.8308873954190097 | Test Accuracy: 0.7737917289486796
Iteration 7 | Training Accuracy: 0.8376080098751886 | Test Accuracy: 0.7802690582959642
Iteration 8 | Training Accuracy: 0.8429570703607188 | Test Accuracy: 0.7867463876432487
Iteration 9 | Training Accuracy: 0.8480318200521191 | Test Accuracy: 0.7887394120577977
Iteration 10 | Training Accuracy: 0.8509120833904814 | Test Accuracy: 0.7897359242650722
Iteration 11 | Training Accura

Iteration 93 | Training Accuracy: 0.9806610890138527 | Test Accuracy: 0.9038365719980069
Iteration 94 | Training Accuracy: 0.9806610890138527 | Test Accuracy: 0.9038365719980069
Iteration 95 | Training Accuracy: 0.9806610890138527 | Test Accuracy: 0.9043348281016442
Iteration 96 | Training Accuracy: 0.9806610890138527 | Test Accuracy: 0.9048330842052815
Iteration 97 | Training Accuracy: 0.9809353998079824 | Test Accuracy: 0.9048330842052815
Iteration 98 | Training Accuracy: 0.9807982444109176 | Test Accuracy: 0.9048330842052815
Iteration 99 | Training Accuracy: 0.9807982444109176 | Test Accuracy: 0.9053313403089188


In [670]:
test=None
if test:
    print("test")

In [498]:
NN.feedforward(X_train)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in exp


array([[  2.71312537e-09,   1.88792083e-18,   2.56342138e-23, ...,
          7.20961547e-09,   1.00000000e+00,   6.66001531e-29],
       [  1.53059479e-28,   1.75232593e-20,   6.38859710e-32, ...,
          2.12023081e-23,   1.03657748e-41,   4.82482829e-05],
       [  3.65769523e-12,   7.37669141e-07,   2.14328249e-27, ...,
          2.91136723e-18,   1.17539040e-17,   8.24125668e-11],
       ..., 
       [  3.51050106e-44,   4.16771004e-21,   7.38725341e-31, ...,
          1.17311509e-20,   6.85843159e-36,   8.57527208e-23],
       [  5.12691575e-29,   5.62544447e-23,   7.67575998e-25, ...,
          6.07369836e-01,   2.32138719e-23,   2.30585616e-02],
       [  5.51519053e-23,   5.77968633e-15,   1.19604969e-13, ...,
          2.13177786e-05,   9.63112125e-13,   4.51834623e-14]])

In [343]:
NN.biases[0].shape

(30, 7291)

In [724]:
(np.abs((y_train-NN.predict(X_train))).sum(axis=1)/2).mean()

0.80839391030037033

In [658]:
pred = NN.predict(X_test)
#from sklearn.metrics import confusion_matrix
#confusion_matrix(pred,y_test)
pred


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:82: RuntimeWarning: overflow encountered in exp


array([9, 4, 3, ..., 4, 0, 1])

In [ ]:
np.argmax(pred,axis=1).shape

In [ ]:
sizes = [4,3,4]
list(zip(sizes[:-1], sizes[1:]))

In [ ]:
list(range(5,0,-1))

In [ ]:
np.concatenate((np.random.randn(2, 3),np.ones((2,1))),axis=1)

In [182]:
X_train[1].shape

(256,)

In [201]:
NN.biases

[array([ 1.00991919, -0.36287491, -0.52331782,  1.68611696, -0.55968326,
         0.31156315,  1.15918928, -0.64807042, -0.19311074, -0.45522268])]

In [221]:

    def _backprop(self,X,y):
        
        #feedforward
        activation = X
        activations = [X]
        local_derivs = []
        
        w = self.weights
        b = self.biases
        for i in range(self.num_layers-1):
            activation = np.dot(w[i],activation.T).T + b[i]
            local_derivs.append(activation)
            activation = self.sigmoid(activation)
            activations.append(activation) #braucht man das?
        
        #calculate loss
        #print(activations[-1])
        #print("Train Err: ", self.error_rate(np.argmax(activations[-1],axis=1),y))
        #print(np.argmax(activations[-1],axis=1))
        
        cost_deriv = y - activations[-1]
        #print(cost_deriv.shape)
        #for i in range(cost_deriv.shape[0]):
        #    cost_deriv[i,int(y[i])] -= 1
        
        
        #backwards
        deltas_b = [np.zeros(b.shape) for b in self.biases]
        deltas_w = [np.zeros(w.shape) for w in self.weights]
        
        delta = cost_deriv * self.sigmoid_deriv(local_derivs[-1])
        #print(delta)
        deltas_b[-1] = np.sum(delta,axis=0)
        deltas_w[-1] = np.dot(delta.T, activations[-2])
        
        for i in range(2,self.num_layers):
            local_deriv = self.sigmoid_deriv(local_derivs[-i])
            delta = np.dot(self.weights[-i+1].T, delta.T) * local_deriv.T
            print(delta)
            deltas_b[-i] = np.sum(delta.T,axis=0)
            deltas_w[-i] = np.dot(delta, activations[-i-1])
            
        #print(deltas_b)
        #print(deltas_w)
        return (deltas_b, deltas_w)

            

IndentationError: unexpected indent (<ipython-input-221-cdcd56c925eb>, line 2)

In [258]:
class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""

        training_data = list(training_data)
        n = len(training_data)

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [286]:
net = Network([256,30,10])

In [287]:
x = X_train.T
y = y_train
b,w = net.backprop(x,y)

In [294]:
w[0].shape

(30, 256)

In [297]:
net.weights[0].shape

(30, 256)

In [279]:
class NeuronalNetwork(Classifier):
    def __init__(self,sizes):
        #self.sizes = sizes
        #self.layers = []
        #self.weights = []
        #self.biases = []
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    
    def train(self, X, y, iterations=10, learning_rate=0.5):
        lr = learning_rate/X.shape[0]
        for i in range(iterations):
            #print(X[i%X.shape[0]][np.newaxis].shape)
            for j in range(1):#X.shape[0]):
                deltas_b, deltas_w = self.backprop(X,y)#[j][np.newaxis],y[j])
                self.weights = [w-(learning_rate*dw)
                            for w, dw in zip(self.weights, deltas_w)] 
                self.biases = [(b-(learning_rate*db) )
                            for b, db in zip(self.biases, deltas_b)]
    
    def backprop(self,x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x.T
        activations = [x.T] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = np.sum(delta.T,axis=0)[np.newaxis].T
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = np.sum(delta.T,axis=0)[np.newaxis].T
            #print(nabla_b[-l].shape)
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""

        output_activations[y.astype(int)] -= 1

        return (output_activations) 
    
    def sigmoid(self,z):
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoid_deriv(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
        
    def feedforward(self,X):
        a = X.T
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a)+b)
        return a
    
    def predict(self,X):
        return np.argmax(self.feedforward(X),axis=0)
    
    def predict_error(self,X,y):
        return self.error_rate(self.predict(X),y)
    

In [281]:
net = MyNetwork([256,30,10])
net.backprop(X_train.T,y_train)

AttributeError: 'MyNetwork' object has no attribute 'cost_derivative'

In [489]:
y_train

array([ 6.,  5.,  4., ...,  3.,  0.,  1.])